In [20]:
import hashlib
import hmac
import base64
import urllib.parse as urlparse
import requests

from PIL import Image

In [40]:
API_key = "nono :)"
secret = "nono :)"

def sign_url(input_url=None, secret=None) -> str:
    # https://developers.google.com/maps/documentation/maps-static/digital-signature
    if not input_url or not secret:
        raise Exception("Both input_url and secret are required")

    url = urlparse.urlparse(input_url)
    url_to_sign = url.path + "?" + url.query
    decoded_key = base64.urlsafe_b64decode(secret)
    signature = hmac.new(decoded_key, str.encode(url_to_sign), hashlib.sha1)
    encoded_signature = base64.urlsafe_b64encode(signature.digest())
    original_url = url.scheme + "://" + url.netloc + url.path + "?" + url.query

    return original_url + "&signature=" + encoded_signature.decode()

def get_sat_image(lat, lon, zoom=15, secret=None) -> requests.Response:
    if not secret:
        raise Exception("Secret is required")
    
    req_url = f"https://maps.googleapis.com/maps/api/staticmap?center={lat},{lon}&zoom={zoom}&size=400x400&maptype=satellite&key={API_key}"
    signed_url = sign_url(req_url, secret)

    response = requests.get(signed_url)
    return response

    
def save_sat_image(response, filename="map.png") -> None:
    if response.status_code != 200:
        raise Exception(f"Failed to get image, got status code {response.status_code}")
    
    with open(filename, "wb") as f:
        f.write(response.content)


def rotate_image(image_path, angle) -> None:
    image_obj = Image.open(image_path)
    rotated_image = image_obj.rotate(angle)
    rotated_image.save(image_path)
    

if __name__ == "__main__":
    # 55.780001, 9.717275
    
    lat = 55.780001
    lon = 9.717275
    zoom = 18
    img_base_path = "./imgs/"

    # for z in range(10, 20):
    #     zoom = z
    #     print(f"Getting image for zoom level {zoom}")
    #     response = get_sat_image(lat, lon, zoom, secret)
    #     save_sat_image(response, f"{img_base_path}map_z{zoom}.png")

    response = get_sat_image(lat, lon, zoom, secret)
    save_sat_image(response)

    rotate_image("map.png", 45)


